<a href="https://colab.research.google.com/github/valesierrai/Introduccion-a-la-Inteligencia-de-Negocios/blob/main/Parcial_1_VSI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**0. Cargamos Librerias**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

Mounted at /content/drive


##**a.	Un restaurante de comidas saludables que solo vende domicilio**

**Tabla 1:**  Estará toda la información relacionada al restaurante como tal, y se llama **‘Info_Restaurante’**.

nombre_restaurante: Es el nombre del restaurante

num_empleados: El número de empleados con los que cuenta actualmente el

restauranre locacion: Ciudad de Colombia donde se encuentran ubicados

ganancias_men: Ganancias mensuales del restaurante en USD

**Tabla 2:**  Esta tabla incluirá toda la información del restaurante con respecto a los domicilios, se llama **‘Info_Domicilios’**

nombre_restaurante: Es el nombre del restaurante

aplicaciones: Número de aplicaciones que utiliza.

aplicación_frecuente: Nombre de la aplicación por donde recibe más pedidos.

pedidos_dia_app: Por medio de aplicaciones.

domicilios_mes: Cantidad de domicilios entregados al mes.

domiciliarios: Cantidad de domiciliarios propios.


In [ ]:
#Cargamos la base de datos
datos_a = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/IB/Tablas_Restaurante.xlsx',
                      sheet_name= None, header= 0 )

In [ ]:
#Miramos el nombre de cada tabla
datos_a.keys()

dict_keys(['Info_Restaurante', 'Info_Domicilios'])

In [ ]:
#Se vincula con sqlite
conexion = sqlite3.connect('ConsultaRestaurante.db')
cursor = conexion.cursor()

In [ ]:
#Dado que se tienen varias tablas, usaremos un ciclo para pasar la información desde el diccionario hasta la base de datos creada
for hoja, datos_a in datos_a.items():
  datos_a.to_sql(hoja, conexion, if_exists = 'replace', index = False)

In [ ]:
#Confirmamos los cambios ejecutados en la base de datos
conexion.commit()

*Preguntas:*
1. ¿Cuál es el nombre del restaurante junto con su locación, que reciben pedidos de Rappi con frecuencia?
2. ¿Nombre y número de empleados de los restaurantes que no están en Bogotá?
3. ¿Nombre de los restaurantes en forma descendente, que tienen una cantidad inferior a 500 domicilios al mes, junto con la cantidad de aplicaciones que utiliza?
4. ¿Cuáles son las aplicaciones que no reciben domicilios de Rappi con frecuencia, que están ubicados en Bogotá?

In [ ]:
#Creamos la consultas (formato de texto), en SQL
#Para visualizar los nombres de todas las tablas que tiene la base de datos
consulta = '''SELECT name
           FROM sqlite_master
           WHERE type = 'table' '''

#Ejecutamos la consulta
cursor.execute(consulta)

#Mostramos los resultados
print(cursor.fetchall()) #fetchall: Se trae toda la información de la base de datos para visualizar

[('Info_Restaurante',), ('Info_Domicilios',)]


In [ ]:
#1. ¿Cuál es el nombre del restaurante junto con su locación, que reciben pedidos de Rappi con frecuencia?
consulta1= '''
SELECT IR.nombre_restaurante,
       IR.locacion,
       ID.aplicación_frecuente
       FROM Info_Restaurante IR
       JOIN Info_Domicilios ID ON IR.nombre_restaurante = ID.nombre_restaurante
       WHERE ID.aplicación_frecuente = 'Rappi'
'''

#Ejecutamos la consulta
cursor.execute(consulta1)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Sano y Rico', 'Medellín', 'Rappi')
('Vital', 'Barranquilla', 'Rappi')
('Verde y Vivo', 'Bucaramanga', 'Rappi')
('Jardin Comestible', 'Bogotá', 'Rappi')
('Vivir Sano', 'Tolima', 'Rappi')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta1 = pd.DataFrame(resultados, columns = columnas)
df_consulta1

,nombre_restaurante,locacion,aplicación_frecuente
0,Sano y Rico,Medellín,Rappi
1,Vital,Barranquilla,Rappi
2,Verde y Vivo,Bucaramanga,Rappi
3,Jardin Comestible,Bogotá,Rappi
4,Vivir Sano,Tolima,Rappi


In [ ]:
#2. ¿Nombre y número de empleados de los restaurantes que no están en Bogotá?
consulta2= '''
SELECT nombre_restaurante,
       num_empleados,
       locacion
       FROM Info_Restaurante
       WHERE locacion != 'Bogotá'
'''

#Ejecutamos la consulta
cursor.execute(consulta2)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Sano y Rico', 25, 'Medellín')
('Vital', 7, 'Barranquilla')
('Verde y Vivo', 12, 'Bucaramanga')
('Sabor Saludable', 9, 'Bucaramanga')
('Vivir Sano', 5, 'Tolima')
('Tierra Madre ', 9, 'Medellín')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta2 = pd.DataFrame(resultados, columns = columnas)
df_consulta2

,nombre_restaurante,num_empleados,locacion
0,Sano y Rico,25,Medellín
1,Vital,7,Barranquilla
2,Verde y Vivo,12,Bucaramanga
3,Sabor Saludable,9,Bucaramanga
4,Vivir Sano,5,Tolima
5,Tierra Madre,9,Medellín


In [ ]:
#3. ¿Nombre de los restaurantes en forma descendente, que tienen una cantidad inferior a 500 domicilios al mes, junto con la cantidad de aplicaciones que utiliza?
consulta3= '''
SELECT nombre_restaurante,
       domicilios_mes,
       aplicaciones
       FROM Info_Domicilios
       WHERE domicilios_mes < 500
       ORDER BY domicilios_mes DESC
'''

#Ejecutamos la consulta
cursor.execute(consulta3)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Equilibrio ', 450, 8)
('Verde y Vivo', 360, 2)


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta3 = pd.DataFrame(resultados, columns = columnas)
df_consulta3

,nombre_restaurante,domicilios_mes,aplicaciones
0,Equilibrio,450,8
1,Verde y Vivo,360,2


In [ ]:
#4. ¿Cuáles son las aplicaciones que no reciben domicilios de Rappi con frecuencia, que están ubicados en Bogotá?
consulta4= '''
SELECT IR.nombre_restaurante,
       IR.locacion,
       ID.aplicación_frecuente
       FROM Info_Restaurante IR
       JOIN Info_Domicilios ID ON IR.nombre_restaurante = ID.nombre_restaurante
       WHERE ID.aplicación_frecuente != 'Rappi' AND IR.locacion = 'Bogotá'
'''
#Ejecutamos la consulta
cursor.execute(consulta4)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Nutrete ', 'Bogotá', 'iFood')
('Equilibrio ', 'Bogotá', 'iFood')
('Hoja Verde', 'Bogotá', 'Uber Eats')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta4 = pd.DataFrame(resultados, columns = columnas)
df_consulta4

,nombre_restaurante,locacion,aplicación_frecuente
0,Nutrete,Bogotá,iFood
1,Equilibrio,Bogotá,iFood
2,Hoja Verde,Bogotá,Uber Eats


##**b. Un torneo profesional de una deporte (voleibol)**

Variables seleccionadas para el torneo profesional de voleibol:

**Tabla 1:** Muestra el nombre del quipo, junto a la información de este con respecto a este torneo, y por eso se llamara **‘Información_Torneo’**.

nombre_equipo: En esta columna se encuentra el nombre del equipo profesional

clasificación: Se muestra el lugar en el que quedaron, cuando se clasificaron al torneo.

torneos_ganados: La cantidad de torneos profesionales que han ganado

torneos_participados: En cuantos torneos ha participado el equipo

puesto_torneo_pasado: En qué lugar quedó el equipo en el torneo anterior (si no clasificó aparecerá NC).

partidos_ganados: Los partidos ganados durante el último año.

**Tabla 2:** Muestra con base al nombre del equipo, información del equipo como tal, llamándose **‘Informacion_Equipo’**.

nombre_equipo: En esta columna se encuentra el nombre del equipo profesional

país: Muestra el país de procedencia del equipo

numero_asistentes: Con cuantos asistentes técnicos cuenta el equipo

entrenador: Nombre completo del entrenador actual del equipo

experiencia: Cuantos años lleva el equipo formado (no el actual sino el club como tal)

uniforme_oficial: El color del uniforme principal del equipo

In [ ]:
#Cargamos la base de datos
datos_b = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/IB/Tablas_Torneo.xlsx',
                      sheet_name= None, header= 0 )

In [ ]:
#Miramos el nombre de cada tabla
datos_b.keys()

dict_keys(['Informacion_Torneo', 'Informacion_Equipo'])

In [ ]:
#Se vincula con sqlite
conexion = sqlite3.connect('ConsultaTorneo.db')
cursor = conexion.cursor()

In [ ]:
#Dado que se tienen varias tablas, usaremos un ciclo para pasar la información desde el diccionario hasta la base de datos creada
for hoja, datos_b in datos_b.items():
  datos_b.to_sql(hoja, conexion, if_exists = 'replace', index = False)

In [ ]:
#Confirmamos los cambios ejecutados en la base de datos
conexion.commit()

*Preguntas:*
1. ¿Cuáles son los nombres, y clasificación de los equipos que van a participar en este torneo, que todavía no lo han ganado?
2. ¿Cuál es el nombre del entrenador, el nombre del equipo, y el uniforme oficial de los equipos que tienen más de 25 años de experiencia?
3. ¿Cuáles es el orden de los equipos clasificados al torneo? Y su país de residencia?
4.  ¿Cuál es el color del uniforme de equipos del torneo pasado que estén participando?

In [ ]:
#Creamos la consultas (formato de texto), en SQL
#Para visualizar los nombres de todas las tablas que tiene la base de datos
consulta = '''SELECT name
           FROM sqlite_master
           WHERE type = 'table' '''

#Ejecutamos la consulta
cursor.execute(consulta)

#Mostramos los resultados
print(cursor.fetchall()) #fetchall: Se trae toda la información de la base de datos para visualizar

[('Informacion_Equipo’',), ('Informacion_Torneo',), ('Informacion_Equipo',)]


In [ ]:
#1. ¿Cuáles son los nombres, y clasificación de los equipos que van a participar en este torneo, que todavía no lo han ganado?
consulta1= '''
SELECT nombre_equipo,
       clasificación
       FROM Informacion_Torneo
       WHERE torneos_ganados == 0
'''

#Ejecutamos la consulta
cursor.execute(consulta1)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Las Leonas', 7)
('Las Aguilas ', 2)
('Las Fénix', 5)


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta1 = pd.DataFrame(resultados, columns = columnas)
df_consulta1

,nombre_equipo,clasificación
0,Las Leonas,7
1,Las Aguilas,2
2,Las Fénix,5


In [ ]:
#2. ¿Cuál es el nombre del entrenador, el nombre del equipo, y el uniforme oficial de los equipos que tienen más de 25 años de experiencia?
consulta2= '''
SELECT entrenador,
       nombre_equipo,
       uniforme_oficial
       FROM Informacion_Equipo
       Where experiencia > 25
'''

#Ejecutamos la consulta
cursor.execute(consulta2)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Antônio Santos', 'Las Aguilas ', 'amarillo')
('Maria Nowak', 'Las Fénix', 'blanco')
('Emily Davis', 'Las Cazadoras', 'azul')
('Olivia Anderson', 'Las Amazonas', 'azul')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta2 = pd.DataFrame(resultados, columns = columnas)
df_consulta2

,entrenador,nombre_equipo,uniforme_oficial
0,Antônio Santos,Las Aguilas,amarillo
1,Maria Nowak,Las Fénix,blanco
2,Emily Davis,Las Cazadoras,azul
3,Olivia Anderson,Las Amazonas,azul


In [ ]:
#3. ¿Cuáles es el orden de los equipos clasificados al torneo? Y su país de residencia
consulta3= '''
SELECT IE.nombre_equipo,
       IT.clasificación,
       IE.país
       FROM Informacion_Torneo IT
       JOIN 'Informacion_Equipo' IE ON IT.nombre_equipo = IE.nombre_equipo
       ORDER BY clasificación ASC
'''

#Ejecutamos la consulta
cursor.execute(consulta3)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Las Tigresas', 1, 'República Dominicana')
('Las Aguilas ', 2, 'Brasil ')
('Las Guerreras ', 3, 'Turquia')
('Las Amazonas', 4, 'Estados Unidos')
('Las Fénix', 5, 'Polonia ')
('Las Volcanicas', 6, 'Italia ')
('Las Leonas', 7, 'Brasil')
('Las Destructuras de Redes', 8, 'Brasil ')
('Las Cazadoras', 9, 'Estados Unidos')
('Las Maestras del Juego', 10, 'República Dominicana')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta3 = pd.DataFrame(resultados, columns = columnas)
df_consulta3

,nombre_equipo,clasificación,país
0,Las Tigresas,1,República Dominicana
1,Las Aguilas,2,Brasil
2,Las Guerreras,3,Turquia
3,Las Amazonas,4,Estados Unidos
4,Las Fénix,5,Polonia
5,Las Volcanicas,6,Italia
6,Las Leonas,7,Brasil
7,Las Destructuras de Redes,8,Brasil
8,Las Cazadoras,9,Estados Unidos
9,Las Maestras del Juego,10,República Dominicana


In [ ]:
#4. ¿Cuál es el color del uniforme de equipos del torneo pasado que estén participando?
consulta4= '''
SELECT IE.nombre_equipo,
       IE.uniforme_oficial,
       IT.puesto_torneo_pasado
       FROM Informacion_Torneo IT
       JOIN 'Informacion_Equipo' IE ON IT.nombre_equipo = IE.nombre_equipo
       WHERE puesto_torneo_pasado != 'NC'
'''

#Ejecutamos la consulta
cursor.execute(consulta4)

#Mostramos los resultados
resultados = cursor.fetchall()
for resultado in resultados:
  print(resultado)

('Las Tigresas', 'vinotinto', '3')
('Las Volcanicas', 'blanco', '4')
('Las Aguilas ', 'amarillo', '6')
('Las Fénix', 'blanco', '8')
('Las Destructuras de Redes', 'verde', '8')
('Las Maestras del Juego', 'vinotinto', '7')
('Las Amazonas', 'azul', '1')


In [ ]:
#Mostramos la tabla resultante
columnas = [descripcion[0] for descripcion in cursor.description]
df_consulta4 = pd.DataFrame(resultados, columns = columnas)
df_consulta4

,nombre_equipo,uniforme_oficial,puesto_torneo_pasado
0,Las Tigresas,vinotinto,3
1,Las Volcanicas,blanco,4
2,Las Aguilas,amarillo,6
3,Las Fénix,blanco,8
4,Las Destructuras de Redes,verde,8
5,Las Maestras del Juego,vinotinto,7
6,Las Amazonas,azul,1


In [ ]:
#Cerramos conexión con la base de datos
conexion.close()